## Generate key stats on hover

After a recent demonstration of your work to a manager in the e-commerce company, you have received a request to create a dashboard for this stakeholder. Having too many visualizations and too much text is absolutely not allowed.

Instead, they want the dashboard to be highly interactive. They have requested a scatter plot that, when hovered over, will provide some additional key stats in a text box to the right. This should change when hovering over a new point on the scatter plot.

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output
ecom_sales = pd.read_csv('/usr/local/share/datasets/ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/production/repositories/5893/datasets/fdbe0accd2581a0c505dab4b29ebb66cf72a1803/e-comlogo.png'
ecom_country = ecom_sales.groupby('Country')['OrderValue'].agg(['sum', 'count']).reset_index().rename(columns={'count':'Sales Volume', 'sum':'Total Sales ($)'})

# Add the country data to the scatter plot
ecom_scatter = px.scatter(ecom_country, x='Total Sales ($)', y='Sales Volume', 
                          color='Country', width=350, height=400, custom_data=['Country'])
ecom_scatter.update_layout({'legend':dict(orientation='h', y=-0.5,x=1, yanchor='bottom', xanchor='right'), 'margin':dict(l=20, r=20, t=25, b=0)})

app = dash.Dash(__name__)

app.layout = html.Div([
  html.Img(src=logo_link, 
        style={'margin':'30px 0px 0px 0px' }),
  html.H1('Sales breakdowns'),
  html.Div(
    children=[
    html.Div(
        children=[
          html.H2('Sales by Country'),
          dcc.Graph(id='scatter_fig', figure=ecom_scatter)
        ],
        style={'width':'350px', 'height':'500px', 'display':'inline-block', 
               'vertical-align':'top', 'border':'1px solid black', 'padding':'20px'}),
    html.Div(
        children=[
          html.H2('Key Stats'),
          html.P(id='text_output', style={'width':'500px', 'text-align':'center'}),
          ],
          style={'width':'700px', 'height':'650px','display':'inline-block'}),
    ]),], 
  style={'text-align':'center', 'display':'inline-block', 'width':'100%'})

# Trigger callback on hover
@app.callback(
    Output('text_output', 'children'),
    Input('scatter_fig', 'hoverData'))

def get_key_stats(hoverData):
    
    if not hoverData:
        return 'Hover over a country to see key stats'
	
    # Extract the custom data from the hoverData
    country = hoverData['points'][0]['customdata'][0]
    country_df = ecom_sales[ecom_sales['Country'] == country]
    
    top_major_cat = country_df.groupby('Major Category').agg('size').reset_index(name='Sales Volume').sort_values(by='Sales Volume', ascending=False).reset_index(drop=True).loc[0,'Major Category']
    top_sales_month = country_df.groupby('Year-Month')['OrderValue'].agg('sum').reset_index(name='Total Sales ($)').sort_values(by='Total Sales ($)', ascending=False).reset_index(drop=True).loc[0,'Year-Month']
	
    # Use the aggregated variables
    stats_list = [
    f'Key stats for : {country}', html.Br(),
    f'The most popular Major Category by sales volume was: {top_major_cat}', html.Br(),
    f'The highest sales value month was: {top_sales_month}'
    ]
    return stats_list

if __name__ == '__main__':
    app.run_server(debug=True)

## Hover to update another plot

The global e-commerce company is loving the innovation and new life your work is breathing into their tired old reports. A new challenge has been sent your way by the manager you recently built the key-stats hover dashboard for. While those words provided a great snapshot, a picture is worth a thousand words! The manager has asked if you could produce graphs that would show different subsets depending on what is being hovered over.

Your previous experience with using callbacks to filter data and regenerate figures, in combination with your recent work on hover triggering callbacks, should be just the perfect combination.

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output
ecom_sales = pd.read_csv('/usr/local/share/datasets/ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/production/repositories/5893/datasets/fdbe0accd2581a0c505dab4b29ebb66cf72a1803/e-comlogo.png'
ecom_country = ecom_sales.groupby('Country')['OrderValue'].agg(['sum', 'count']).reset_index().rename(columns={'count':'Sales Volume', 'sum':'Total Sales ($)'})
ecom_scatter = px.scatter(ecom_country, x='Total Sales ($)', y='Sales Volume', color='Country', width=350, height=550, custom_data=['Country'])
ecom_scatter.update_layout({'legend':dict(orientation='h', y=-0.7,x=1, yanchor='bottom', xanchor='right')})

app = dash.Dash(__name__)

app.layout = html.Div([
  html.Img(src=logo_link, 
        style={'margin':'30px 0px 0px 0px' }),
  html.H1('Sales breakdowns'),
  html.Div(
    children=[
    html.Div(
        children=[
          html.H3('Sales Volume vs Sales Amount by Country'),
          dcc.Graph(id='scatter', figure=ecom_scatter),
        ],
        style={'width':'350px', 'height':'650px', 'display':'inline-block', 
               'vertical-align':'top', 'border':'1px solid black', 'padding':'20px'}),
    html.Div(
      children=[
        # Add two plot types to the layout
        dcc.Graph(id='major_cat'),
        dcc.Graph(id='minor_cat'),
      ],
      style={'width':'700px', 'height':'650px','display':'inline-block'})
    ]),], 
  style={'text-align':'center', 'display':'inline-block', 'width':'100%'}
  )

# Create a callback to update the minor category plot
@app.callback(
    Output('minor_cat', 'figure'),
    Input('scatter', 'hoverData'))

def update_min_cat_hover(hoverData):
    hover_country = 'Australia'
    
    if hoverData:
        hover_country = hoverData['points'][0]['customdata'][0]

    minor_cat_df = ecom_sales[ecom_sales['Country'] == hover_country]
    minor_cat_agg = minor_cat_df.groupby('Minor Category')['OrderValue'].agg('sum').reset_index(name='Total Sales ($)')
    ecom_bar_minor_cat = px.bar(minor_cat_agg, x='Total Sales ($)', y='Minor Category', orientation='h', height=450, title=f'Sales by Minor Category for: {hover_country}')
    ecom_bar_minor_cat.update_layout({'yaxis':{'dtick':1, 'categoryorder':'total ascending'}, 'title':{'x':0.5}})

    return ecom_bar_minor_cat

# Create a callback to update the major category plot
@app.callback(
    Output('major_cat', 'figure'),
    Input('scatter', 'hoverData'))

def update_major_cat_hover(hoverData):
    hover_country = 'Australia'
    
    # Conditionally select the country from the hover data
    if hoverData:
        hover_country = hoverData['points'][0]['customdata'][0]

    major_cat_df = ecom_sales[ecom_sales['Country'] == hover_country]
    major_cat_agg = major_cat_df.groupby('Major Category')['OrderValue'].agg('sum').reset_index(name='Total Sales ($)')

    ecom_bar_major_cat = px.bar(major_cat_agg, x='Total Sales ($)', 
                                y='Major Category', height=300, 
                                title=f'Sales by Major Category for: {hover_country}', color='Major Category',
            color_discrete_map={'Clothes':'blue','Kitchen':'red', 'Garden':'green', 'Household':'yellow'})
    ecom_bar_major_cat.update_layout({'margin':dict(l=10,r=15,t=40,b=0), 'title':{'x':0.5}})

    return ecom_bar_major_cat

if __name__ == '__main__':
    app.run_server(debug=True)

## Click to update another plot

The hover-to-update plot was very well received. What magic! Your work is making experiences out of tired, static visuals.

There has been a request to change the dashboard you created in the last lesson. The reporting hierarchy of the e-commerce company begins at country, then major category, then minor category. Therefore, they would prefer if the hover didn't update the bottom bar chart.

Instead, they wonder if you could click on the major category bar chart and have that update the bar chart below. This has been requested because there is a desire to understand how the proportion of sales by major category differs between countries.

From your recent work in clicks and hovers, you know just the way to implement it!

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output
ecom_sales = pd.read_csv('/usr/local/share/datasets/ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/production/repositories/5893/datasets/fdbe0accd2581a0c505dab4b29ebb66cf72a1803/e-comlogo.png'
ecom_country = ecom_sales.groupby('Country')['OrderValue'].agg(['sum', 'count']).reset_index().rename(columns={'count':'Sales Volume', 'sum':'Total Sales ($)'})
ecom_scatter = px.scatter(ecom_country, x='Total Sales ($)', y='Sales Volume', color='Country', width=350, height=550, custom_data=['Country'])
ecom_scatter.update_layout({'legend':dict(orientation='h', y=-0.7,x=1, yanchor='bottom', xanchor='right')})

app = dash.Dash(__name__)

app.layout = html.Div([
  html.Img(src=logo_link, 
        style={'margin':'30px 0px 0px 0px' }),
  html.H1('Sales breakdowns'),
  html.Div(
    children=[
    html.Div(
        children=[
          html.H3('Sales Volume vs Sales Amount by Country'),
          dcc.Graph(id='scatter', figure=ecom_scatter),
        ],
        style={'width':'350px', 'height':'650px', 'display':'inline-block', 
               'vertical-align':'top', 'border':'1px solid black', 'padding':'20px'}),
    html.Div(
      children=[
        dcc.Graph(id='major_cat'),
        dcc.Graph(id='minor_cat'),
      ],
      style={'width':'700px', 'height':'650px','display':'inline-block'})
    ]),], 
  style={'text-align':'center', 'display':'inline-block', 'width':'100%'}
  )

@app.callback(
    Output('major_cat', 'figure'),
    Input('scatter', 'hoverData'))

def update_major_cat_hover(hoverData):
    hover_country = 'Australia'
    
    if hoverData:
        hover_country = hoverData['points'][0]['customdata'][0]

    major_cat_df = ecom_sales[ecom_sales['Country'] == hover_country]
    major_cat_agg = major_cat_df.groupby('Major Category')['OrderValue'].agg('sum').reset_index(name='Total Sales ($)')

    ecom_bar_major_cat = px.bar(major_cat_agg, x='Total Sales ($)',
                                # Ensure the Major category will be available
                                custom_data=['Major Category'], 
                                y='Major Category', height=300, 
                                title=f'Sales by Major Category for: {hover_country}', color='Major Category',
            color_discrete_map={'Clothes':'blue','Kitchen':'red', 'Garden':'green', 'Household':'yellow'})
    ecom_bar_major_cat.update_layout({'margin':dict(l=10,r=15,t=40,b=0), 'title':{'x':0.5}})

    return ecom_bar_major_cat

# Set up a callback for click data
@app.callback(
    Output('minor_cat', 'figure'),
    Input('major_cat', 'clickData'))

def update_major_cat_click(clickData):
    click_cat = 'All'
    major_cat_df = ecom_sales.copy()
    total_sales = major_cat_df.groupby('Country')['OrderValue'].agg('sum').reset_index(name='Total Sales ($)')
    
    # Extract the major category clicked on for usage
    if clickData:
        click_cat = clickData['points'][0]['customdata'][0]
        
        # Undetake a filter using the major category clicked on
        major_cat_df = ecom_sales[ecom_sales['Major Category'] == click_cat]
    
    country_mj_cat_agg = major_cat_df.groupby('Country')['OrderValue'].agg('sum').reset_index(name='Total Sales ($)')
    country_mj_cat_agg['Sales %'] = (country_mj_cat_agg['Total Sales ($)'] / total_sales['Total Sales ($)'] * 100).round(1)
    
    ecom_bar_country_mj_cat = px.bar(country_mj_cat_agg, x='Sales %', y='Country', 
                                orientation='h', height=450, range_x = [0,100], text='Sales %', 
                                     title=f'Global Sales % by Country for Major Category: {click_cat}')
    ecom_bar_country_mj_cat.update_layout({'yaxis':{'dtick':1, 'categoryorder':'total ascending'}, 'title':{'x':0.5}})

    return ecom_bar_country_mj_cat  
  

if __name__ == '__main__':
    app.run_server(debug=True)

## Conditional dropdown options

A senior purchasing officer in the e-commerce company has posed an interesting problem. Their existing dashboard, showing sales by month for minor categories of items, has a two-level dropdown (major category and minor category) with way too many options in it. It is especially annoying, you are told, that some options appear in the second dropdown that can not be selected.

You assure the stakeholder you can assist - it is a great opportunity for you to use chained callbacks to produce a conditional dropdown.

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output
ecom_sales = pd.read_csv('/usr/local/share/datasets/ecom_sales.csv')
major_categories = list(ecom_sales['Major Category'].unique())
minor_categories = list(ecom_sales['Minor Category'].unique())
logo_link = 'https://assets.datacamp.com/production/repositories/5893/datasets/fdbe0accd2581a0c505dab4b29ebb66cf72a1803/e-comlogo.png'
ecom_country = ecom_sales.groupby('Country')['OrderValue'].agg(['sum', 'count']).reset_index().rename(columns={'count':'Sales Volume', 'sum':'Total Sales ($)'})

app = dash.Dash(__name__)

app.layout = html.Div([
  html.Img(src=logo_link, 
        style={'margin':'30px 0px 0px 0px' }),
  html.H1('Sales breakdowns'),
  html.Div(
    children=[
    html.Div(
      children=[
      html.H2('Controls'),
      html.Br(),
      html.H3('Major Category Select'),
      dcc.Dropdown(
        id='major_cat_dd',
        # Set up the Major Category options with the same label and value
        options=[{'label':category, 'value':category} for category in major_categories],
      style={'width':'200px', 'margin':'0 auto'}),
      html.Br(),
      html.H3('Minor Category Select'),
      dcc.Dropdown(
        id='minor_cat_dd',
        style={'width':'200px', 'margin':'0 auto'})
        ],
        style={'width':'350px', 'height':'350px', 'display':'inline-block', 
               'vertical-align':'top', 'border':'1px solid black', 'padding':'20px'}),
    html.Div(
      children=[
      dcc.Graph(id='sales_line')],
      style={'width':'700px', 'height':'650px','display':'inline-block'})
    ]),], 
  style={'text-align':'center', 'display':'inline-block', 'width':'100%'})


# Create a callback from the Major Category dropdown to the Minor Category Dropdown
@app.callback(
    Output('minor_cat_dd', 'options'),
    Input('major_cat_dd', 'value'))

def update_minor_dd(major_cat_dd):
  
    major_minor = ecom_sales[['Major Category', 'Minor Category']].drop_duplicates()
    relevant_minor_options = major_minor[major_minor['Major Category'] == major_cat_dd]['Minor Category'].values.tolist()
    
    # Create and return formatted relevant options with the same label and value
    formatted_relevant_minor_options = [{'label':x, 'value':x} for x in relevant_minor_options]
    return formatted_relevant_minor_options

# Create a callback for the Minor Category dropdown to update the line plot
@app.callback(
    Output('sales_line', 'figure'),
    Input('minor_cat_dd', 'value'))

def update_line(minor_cat):
    minor_cat_title = 'All'
    ecom_line = ecom_sales.copy()
    
    if minor_cat:
        minor_cat_title = minor_cat
        ecom_line = ecom_line[ecom_line['Minor Category'] == minor_cat]
    
    ecom_line = ecom_line.groupby('Year-Month')['OrderValue'].agg('sum').reset_index(name='Total Sales ($)')
    line_graph = px.line(ecom_line, x='Year-Month',  y='Total Sales ($)', 
                         title=f'Total Sales by Month for Minor Category: {minor_cat_title}')
    
    return line_graph


if __name__ == '__main__':
    app.run_server(debug=True)

## Extending the chain

Your conditional dropdown was a smash hit! That really cleaned up the dashboard and continues to provide a much nicer user experience. When demonstrating this product, the stakeholder asked if you could implement a few updates.

Firstly, they wish to have a default value selected for the minor category, so the graph always has something appearing. You have an idea of how to achieve this by inserting an additional chained callback in between the first two created.

Secondly, they wish to have a title below the graph that reminds users what major category the chosen minor category belongs to. Rather than another callback, you think this can be achieved by sending the chosen major category to an additional output.

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
import random
from dash.dependencies import Input, Output
ecom_sales = pd.read_csv('/usr/local/share/datasets/ecom_sales.csv')
major_categories = list(ecom_sales['Major Category'].unique())
minor_categories = list(ecom_sales['Minor Category'].unique())
logo_link = 'https://assets.datacamp.com/production/repositories/5893/datasets/fdbe0accd2581a0c505dab4b29ebb66cf72a1803/e-comlogo.png'
ecom_country = ecom_sales.groupby('Country')['OrderValue'].agg(['sum', 'count']).reset_index().rename(columns={'count':'Sales Volume', 'sum':'Total Sales ($)'})

app = dash.Dash(__name__)

app.layout = html.Div([
  html.Img(src=logo_link, 
        style={'margin':'30px 0px 0px 0px' }),
  html.H1('Sales breakdowns'),
  html.Div(
    children=[
    html.Div(
      children=[
      html.H2('Controls'),
      html.Br(),
      html.H3('Major Category Select'),
      dcc.Dropdown(
        id='major_cat_dd',
        options=[{'label':category, 'value':category} for category in major_categories],
      style={'width':'200px', 'margin':'0 auto'}),
      html.Br(),
      html.H3('Minor Category Select'),
      dcc.Dropdown(
        id='minor_cat_dd',
        style={'width':'200px', 'margin':'0 auto'})
        ],
        style={'width':'350px', 'height':'350px', 'display':'inline-block', 
               'vertical-align':'top', 'border':'1px solid black', 'padding':'20px'}),
    html.Div(
      children=[
      dcc.Graph(id='sales_line'),
      html.H3(id='chosen_major_cat_title')
      ],
      style={'width':'700px', 'height':'650px','display':'inline-block'})
    ]),], 
  style={'text-align':'center', 'display':'inline-block', 'width':'100%'})

# One callback to set minor values & HTML output
@app.callback(
   Output('minor_cat_dd', 'options'),
   Output('chosen_major_cat_title', 'children'),
   Input('major_cat_dd', 'value'))

def update_minor_dd(major_cat_dd):
  
    major_minor = ecom_sales[['Major Category', 'Minor Category']].drop_duplicates()
    relevant_minor_options = major_minor[major_minor['Major Category'] == major_cat_dd]['Minor Category'].values.tolist()
    minor_options = [{'label':x, 'value':x} for x in relevant_minor_options]
    
    if not major_cat_dd:
        major_cat_dd = 'None Selected'
    # Creating string for title
    major_cat_title = f'This is in the Major Category of : {major_cat_dd}'
    
    # Return the options and title
    return minor_options, major_cat_title

# Create a callback to set a default minor category value
@app.callback(
    Output('minor_cat_dd', 'value'),
    Input('minor_cat_dd', 'options'))

def select_minor_cat(options):
    chosen_val = 'None'
    if options:
        vals = [x['value'] for x in options]
        chosen_val = random.choice(vals)
    return chosen_val
  
@app.callback(
    Output('sales_line', 'figure'),
    Input('minor_cat_dd', 'value'))

def update_line(minor_cat):
    minor_cat_title = 'All'
    ecom_line = ecom_sales.copy()
    
    if minor_cat:
        minor_cat_title = minor_cat
        ecom_line = ecom_line[ecom_line['Minor Category'] == minor_cat]
    
    ecom_line = ecom_line.groupby('Year-Month')['OrderValue'].agg('sum').reset_index(name='Total Sales ($)')
    line_graph = px.line(ecom_line, x='Year-Month',  y='Total Sales ($)', 
                         title=f'Total Sales by Month for Minor Category: {minor_cat_title}')
    
    return line_graph


if __name__ == '__main__':
    app.run_server(debug=True)

## Interactive key stats table

Using graphs to display your data visually is a great way to see aggregated figures at a glance. Yet sometimes, you also need to see more numbers than is feasible to display in common visualization formats.

A key stakeholder in the e-commerce company, the finance department head, has heard of your great work but is someone who needs to see some figures in tabular rather than visual format. He has requested a dashboard displaying some aggregated stats in a tabular format. However, he wonders if you could also include some 'Dash magic' to make it a bit more interactive than his existing Excel spreadsheets?

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash_table import DataTable, FormatTemplate
import pandas as pd
from dash.dependencies import Input, Output
ecom_sales = pd.read_csv('/usr/local/share/datasets/ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/production/repositories/5893/datasets/fdbe0accd2581a0c505dab4b29ebb66cf72a1803/e-comlogo.png'
key_stats_tb = ecom_sales.groupby(['Country','Major Category', 'Minor Category'])['OrderValue'].agg(['sum', 'count', 'mean']).reset_index().rename(columns={'count':'Sales Volume', 'sum':'Total Sales ($)', 'mean':'Average Order Value ($)'})

# Set up money format for relevant columns
money_format = FormatTemplate.money(2)
money_cols = ['Total Sales ($)', 'Average Order Value ($)']
data_cols = [x for x in key_stats_tb.columns if x not in money_cols]
d_columns = [{'name': x, 'id': x} for x in data_cols]
d_columns +=[{'name': x, 'id': x, 
              'type':'numeric', 'format': money_format
  			} for x in money_cols]

# Create a Data Table
d_table = DataTable(
            columns=d_columns,
            data=key_stats_tb.to_dict('records'),
            cell_selectable=False,
            # Add sorting
            sort_action='native',
            # Add filtering
            filter_action='native'
            )

app = dash.Dash(__name__)

app.layout = html.Div([
  html.Img(src=logo_link, 
        style={'margin':'30px 0px 0px 0px' }),
  html.H1('Sales Aggregations'),
  html.Div(
    children=[
      html.H2('Key Aggregations'),
      d_table
      ],
      style={'width':'850px', 'height':'750px', 'margin':'0 auto'}
    ),
    ], 
  style={'text-align':'center', 'display':'inline-block', 'width':'100%'}
  )

if __name__ == '__main__':
    app.run_server(debug=True)

## Paginating a key stats table

The key stats table created in the previous lesson was a great addition to the analytical toolkit of the global e-commerce company. However, it was far too long!

Let's fix that using Dash Data Table's built-in pagination options. This will greatly enhance the usability of the app and allow users to explore different sections of the data as they require.

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash_table import DataTable, FormatTemplate
import pandas as pd
from dash.dependencies import Input, Output
ecom_sales = pd.read_csv('/usr/local/share/datasets/ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/production/repositories/5893/datasets/fdbe0accd2581a0c505dab4b29ebb66cf72a1803/e-comlogo.png'
key_stats_tb = ecom_sales.groupby(['Country','Major Category', 'Minor Category'])['OrderValue'].agg(['sum', 'count', 'mean']).reset_index().rename(columns={'count':'Sales Volume', 'sum':'Total Sales ($)', 'mean':'Average Order Value ($)'})

money_format = FormatTemplate.money(2)
money_cols = ['Total Sales ($)', 'Average Order Value ($)']
data_cols = [x for x in key_stats_tb.columns if x not in money_cols]
d_columns = [{'name': x, 'id': x} for x in data_cols]
d_columns +=[{'name': x, 'id': x, 
              'type':'numeric', 'format': money_format
  			} for x in money_cols]

# Create a Data Table
d_table = DataTable(
            columns=d_columns,
            data=key_stats_tb.to_dict('records'),
            cell_selectable=False,
            sort_action='native',
            filter_action='native',
            # Add pagination
            page_action='native',
            # Start on the first page
            page_current=0,
            # Render 7 items per page
            page_size=7,
            )

app = dash.Dash(__name__)

app.layout = html.Div([
  html.Img(src=logo_link, 
        style={'margin':'30px 0px 0px 0px' }),
  html.H1('Sales Aggregations'),
  html.Div(
    children=[
      html.H2('Key Aggregations'),
      d_table
      ],
      style={'width':'850px', 'height':'750px', 'margin':'0 auto'}
    ),
    ], 
  style={'text-align':'center', 'display':'inline-block', 'width':'100%'}
  )

if __name__ == '__main__':
    app.run_server(debug=True)

## Enhancing the sales dashboard

The e-commerce company you have been working with has been loving the work you have produced so far. Before working with you, they didn't know what was possible, and now they have some complex and creative requests for you!

They have requested an epic dashboard with a laundry list of functionality. Luckily, most of this can be drawn from your previous work with them. There is just a bit of plumbing to do to tidy it up and pull it all together.

Now they wish to have:

A line chart of the total sales by month
That can be filtered by a major and minor category dropdown
Selecting a major category should subset the minor category options to only relevant ones
A table of key stats for major and minor categories below this
A bar chart of sales by country
Hovering this should filter the bar chart by minor category
A bar chart of sales by minor category

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash_table import DataTable, FormatTemplate
import pandas as pd
from dash.dependencies import Input, Output
ecom_sales = pd.read_csv('/usr/local/share/datasets/ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/production/repositories/5893/datasets/fdbe0accd2581a0c505dab4b29ebb66cf72a1803/e-comlogo.png'
major_categories = list(ecom_sales['Major Category'].unique())
large_tb = ecom_sales.groupby(['Major Category', 'Minor Category'])['OrderValue'].agg(['sum', 'count', 'mean']).reset_index().rename(columns={'count':'Sales Volume', 'sum':'Total Sales ($)', 'mean':'Average Order Value ($)'})
ecom_country = ecom_sales.groupby('Country')['OrderValue'].agg('sum').reset_index(name='Total Sales ($)')
bar_fig_country = px.bar(ecom_country, x='Total Sales ($)', y='Country', width=500, height=450, title='Total Sales by Country (Hover to filter the Minor Category bar chart!)', custom_data=['Country'], color='Country', color_discrete_map={'United Kingdom':'lightblue', 'Germany':'orange', 'France':'darkblue', 'Australia':'green', 'Hong Kong':'red'})

money_format = FormatTemplate.money(2)
money_cols = ['Total Sales ($)', 'Average Order Value ($)']
d_columns = [{'name':x, 'id':x} for x in large_tb.columns if x not in money_cols]
d_columns += [
    {'name':'Total Sales ($)', 'id':'Total Sales ($)', 
    'type':'numeric', 
    'format':money_format
    },
    {'name':'Average Order Value ($)', 'id':'Average Order Value ($)', 
    'type':'numeric', 
    'format':money_format}]

d_table = DataTable(
  			# Set up the columns and data
            columns=d_columns,
            data=large_tb.to_dict('records'),
            cell_selectable=False,
  			# Set up sort, filter and pagination
            sort_action='native',
            filter_action='native',
            page_action='native',
            page_current= 0,
            page_size= 7,
            )

app = dash.Dash(__name__)

app.layout = html.Div([
  html.Img(src=logo_link, 
        style={'margin':'30px 0px 0px 0px' }),
  html.H1('Sales breakdowns'),
  html.Div(
    children=[
    html.Div(
        children=[
        html.H2('Controls'),
        html.Br(),
        html.H3('Major Category Select'),
        dcc.Dropdown(id='major_cat_dd',
        options=[{'label':category, 'value':category} for category in major_categories],
            style={'width':'200px', 'margin':'0 auto'}),
        html.Br(),
        html.H3('Minor Category Select'),
        dcc.Dropdown(id='minor_cat_dd',
            style={'width':'200px', 'margin':'0 auto'})
        ],
        style={'width':'350px', 'height':'360px', 'display':'inline-block', 'vertical-align':'top', 'border':'1px solid black', 'padding':'20px'}),
    html.Div(children=[
            html.H3(id='chosen_major_cat_title'),
            dcc.Graph(id='sales_line')
            ],
             style={'width':'700px', 'height':'380px','display':'inline-block', 'margin-bottom':'5px'}
             )
    ]),
    html.Div(
      		# Insert the Data Table
            d_table,
        style={'width':'1000px', 'height':'350px', 'margin':'10px auto', 'padding-right':'30px'}),
  html.Div(children=[
      html.Div(dcc.Graph(id='major_cat', figure=bar_fig_country), style={'display':'inline-block'}),
      html.Div(dcc.Graph(id='minor_cat'), style={'display':'inline-block'})
            ],
             style={'width':'1000px', 'height':'650px','display':'inline-block'}
             ),
  ],style={'text-align':'center', 'display':'inline-block', 'width':'100%'}
  )

# Set up a callback with multiple outputs
@app.callback(
   Output('minor_cat_dd', 'options'),
   Output('chosen_major_cat_title', 'children'),
   Input('major_cat_dd', 'value'))

def update_dd(major_cat_dd):
    major_minor = ecom_sales[['Major Category', 'Minor Category']].drop_duplicates()
    relevant_minor = major_minor[major_minor['Major Category'] == major_cat_dd]['Minor Category'].values.tolist()
    minor_options = [dict(label=x, value=x) for x in relevant_minor]

    if not major_cat_dd:
        major_cat_dd = 'ALL'
    
    major_cat_title = f'This is in the Major Category of : {major_cat_dd}'

    return minor_options, major_cat_title

@app.callback(
    Output('sales_line', 'figure'),
    Input('minor_cat_dd', 'value'))

def update_line(minor_cat):
    minor_cat_title = 'All'
    ecom_line = ecom_sales.copy()
    if minor_cat:
        minor_cat_title = minor_cat
        ecom_line = ecom_line[ecom_line['Minor Category'] == minor_cat]
    ecom_line = ecom_line.groupby('Year-Month')['OrderValue'].agg('sum').reset_index(name='Total Sales ($)')
    line_graph = px.line(ecom_line, x='Year-Month',  y='Total Sales ($)', title=f'Total Sales by Month for Minor Category: {minor_cat_title}', height=350)
    
    return line_graph

@app.callback(
    Output('minor_cat', 'figure'),
    Input('major_cat', 'hoverData'))

def update_min_cat_hover(hoverData):
    hover_country = 'Australia'
    
    if hoverData:
        hover_country = hoverData['points'][0]['customdata'][0]

    minor_cat_df = ecom_sales[ecom_sales['Country'] == hover_country]
    minor_cat_agg = minor_cat_df.groupby('Minor Category')['OrderValue'].agg('sum').reset_index(name='Total Sales ($)')
    ecom_bar_minor_cat = px.bar(minor_cat_agg, x='Total Sales ($)', y='Minor Category', orientation='h', height=450, width=480,title=f'Sales by Minor Category for: {hover_country}')
    ecom_bar_minor_cat.update_layout({'yaxis':{'dtick':1, 'categoryorder':'total ascending'}, 'title':{'x':0.5}})

    return ecom_bar_minor_cat


if __name__ == '__main__':
    app.run_server(debug=True)

## Styling a key stats table

The finance department head loved the key stats table you created for them previously. However, being a big fan of Excel, he has requested some additional formatting of the table.

Specifically, he has requested:

All the cell contents should align left
The background of the cells of the two money columns should have a faded yellow background and bright yellow header
The other column headers should have a light blue background.

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash_table import DataTable, FormatTemplate
import pandas as pd
from dash.dependencies import Input, Output
ecom_sales = pd.read_csv('/usr/local/share/datasets/ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/production/repositories/5893/datasets/fdbe0accd2581a0c505dab4b29ebb66cf72a1803/e-comlogo.png'
key_stats_tb = ecom_sales.groupby(['Country','Major Category', 'Minor Category'])['OrderValue'].agg(['sum', 'count', 'mean']).reset_index().rename(columns={'count':'Sales Volume', 'sum':'Total Sales ($)', 'mean':'Average Order Value ($)'})

money_format = FormatTemplate.money(2)
money_cols = ['Total Sales ($)', 'Average Order Value ($)']
data_cols = [x for x in key_stats_tb.columns if x not in money_cols]
d_columns = [{'name': x, 'id': x} for x in data_cols]
d_columns +=[{'name': x, 'id': x, 
              'type':'numeric', 'format': money_format
  			} for x in money_cols]

d_table = DataTable(
            columns=d_columns,
            data=key_stats_tb.to_dict('records'),
            cell_selectable=False,
            sort_action='native',
            filter_action='native',
            page_action='native',
            page_current=0,
            page_size=10,
  			# Align all cell contents left
  			style_cell=({'textAlign':'left'}),
  			# Style the background of money columns
  			style_cell_conditional=[
            {
              'if': 
			{'column_id':'Total Sales ($)'},
              'background-color':'rgb(252, 252, 184)','textAlign':'center'},
              {
              'if': 
			{'column_id':'Average Order Value ($)'},
              'background-color':'rgb(252, 252, 184)','textAlign':'center'}
            ],
 			# Style all headers
  			style_header={'background-color':'rgb(168, 255, 245)'},
  			# Style money header columns
  			style_header_conditional=[
            {
              'if': 
			{'column_id':'Total Sales ($)'},
              'background-color':'rgb(252, 252, 3)'},
              {
              'if': 
			{'column_id':'Average Order Value ($)'},
              'background-color':'rgb(252, 252, 3)'}
            ],
  			
            )

app = dash.Dash(__name__)

app.layout = html.Div([
  html.Img(src=logo_link, 
        style={'margin':'30px 0px 0px 0px' }),
  html.H1('Sales Aggregations'),
  html.Div(
    children=[
      html.H2('Key Aggregations'),
      d_table
      ],
      style={'width':'850px', 'height':'750px', 'margin':'0 auto'}
    ),
    ], 
  style={'text-align':'center', 'display':'inline-block', 'width':'100%'}
  )

if __name__ == '__main__':
    app.run_server(debug=True)

## Interactive sales dashboard tables

The overarching sales dashboard has come such a long way since you began.

The dashboard has been sent far and wide in the company with some very positive reviews. However, there have been some requests to make the tabular component more interactive. There has been a request to add the ability to select a certain column and have this change the axis on the scatter plot below.

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash_table import DataTable, FormatTemplate
import pandas as pd
from dash.dependencies import Input, Output
ecom_sales = pd.read_csv('/usr/local/share/datasets/ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/production/repositories/5893/datasets/fdbe0accd2581a0c505dab4b29ebb66cf72a1803/e-comlogo.png'
major_categories = list(ecom_sales['Major Category'].unique())
large_tb = ecom_sales.groupby(['Country'])['OrderValue'].agg(['sum', 'count', 'mean', 'median']).reset_index().rename(columns={'count':'Sales Volume', 'sum':'Total Sales ($)', 'mean':'Average Order Value ($)', 'median':'Median Order Value ($)'})
ecom_country = ecom_sales.groupby('Country')['OrderValue'].agg('sum').reset_index(name='Total Sales ($)')
bar_fig_country = px.bar(ecom_country, x='Total Sales ($)', y='Country', width=500, height=450, title='Total Sales by Country (Hover to filter the Minor Category bar chart!)', custom_data=['Country'], color='Country', color_discrete_map={'United Kingdom':'lightblue', 'Germany':'orange', 'France':'darkblue', 'Australia':'green', 'Hong Kong':'red'})

money_format = FormatTemplate.money(2)
money_cols = ['Total Sales ($)', 'Average Order Value ($)', 'Median Order Value ($)']
d_columns = [{'name':x, 'id':x} for x in large_tb.columns if x not in money_cols]
d_columns += [
    {'name':'Total Sales ($)', 'id':'Total Sales ($)', 
    'type':'numeric', 
    'format':money_format
     # Allow columns to be selected
    , 'selectable':True
    },
    {'name':'Average Order Value ($)', 'id':'Average Order Value ($)', 
    'type':'numeric', 
    'format':money_format
     # Allow columns to be selected
    , 'selectable':True
    },
    {'name':'Median Order Value ($)', 'id':'Median Order Value ($)', 
    'type':'numeric', 
    'format':money_format
     # Allow columns to be selected
    , 'selectable':True
    }]


d_table = DataTable(
  			id='my_dt',
            columns=d_columns,
            data=large_tb.to_dict('records'),
            cell_selectable=False,
            sort_action='native',
  			# Make single columns selectable
            column_selectable='single'
            )

app = dash.Dash(__name__)

app.layout = html.Div([
  html.Img(src=logo_link, 
        style={'margin':'30px 0px 0px 0px' }),
  html.H1('Sales breakdowns'),
  html.Div(
    children=[
    html.Div(
        children=[
        html.H2('Controls'),
        html.Br(),
        html.H3('Major Category Select'),
        dcc.Dropdown(id='major_cat_dd',
        options=[{'label':category, 'value':category} for category in major_categories],
            style={'width':'200px', 'margin':'0 auto'}),
        html.Br(),
        html.H3('Minor Category Select'),
        dcc.Dropdown(id='minor_cat_dd',
            style={'width':'200px', 'margin':'0 auto'})
        ],
        style={'width':'350px', 'height':'360px', 'display':'inline-block', 'vertical-align':'top', 'border':'1px solid black', 'padding':'20px'}),
    html.Div(children=[
            html.H3(id='chosen_major_cat_title'),
            dcc.Graph(id='sales_line')
            ],
             style={'width':'700px', 'height':'380px','display':'inline-block', 'margin-bottom':'5px'}
             )
    ]),
    html.Div(
            d_table
        , style={'width':'1000px', 'height':'200px', 'margin':'10px auto', 'padding-right':'30px'}),
  html.Div(children=[
      dcc.Graph(id='scatter_compare'),
      html.Div(dcc.Graph(id='major_cat', figure=bar_fig_country), style={'display':'inline-block'}),
      html.Div(dcc.Graph(id='minor_cat'), style={'display':'inline-block'})
            ],
             style={'width':'1000px', 'height':'650px','display':'inline-block'}
             ),
  ],style={'text-align':'center', 'display':'inline-block', 'width':'100%'}
  )

# Create a callback triggered by selecting a column
@app.callback(
    Output('scatter_compare', 'figure'),
    Input('my_dt', 'selected_columns'))

def table_country(selected_columns):
    comparison_col = 'Total Sales ($)'
	
    # Extract comparison col using its index
    if selected_columns:
        comparison_col = selected_columns[0]

    scatter_fig = px.scatter(
        data_frame=large_tb,
        x='Sales Volume',
      	# Use comparison col in figure
        y=comparison_col,
        color='Country',
        title=f'Sales Volume vs {comparison_col} by country'
    )

    return scatter_fig

@app.callback(
   Output('minor_cat_dd', 'options'),
   Output('chosen_major_cat_title', 'children'),
   Input('major_cat_dd', 'value'))

def update_dd(major_cat_dd):
    major_minor = ecom_sales[['Major Category', 'Minor Category']].drop_duplicates()
    relevant_minor = major_minor[major_minor['Major Category'] == major_cat_dd]['Minor Category'].values.tolist()
    minor_options = [dict(label=x, value=x) for x in relevant_minor]

    if not major_cat_dd:
        major_cat_dd = 'ALL'
    
    major_cat_title = f'This is in the Major Category of : {major_cat_dd}'

    return minor_options, major_cat_title

@app.callback(
    Output('sales_line', 'figure'),
    Input('minor_cat_dd', 'value'))

def update_line(minor_cat):
    minor_cat_title = 'All'
    ecom_line = ecom_sales.copy()
    if minor_cat:
        minor_cat_title = minor_cat
        ecom_line = ecom_line[ecom_line['Minor Category'] == minor_cat]
    ecom_line = ecom_line.groupby('Year-Month')['OrderValue'].agg('sum').reset_index(name='Total Sales ($)')
    line_graph = px.line(ecom_line, x='Year-Month',  y='Total Sales ($)', title=f'Total Sales by Month for Minor Category: {minor_cat_title}', height=350)
    
    return line_graph

@app.callback(
    Output('minor_cat', 'figure'),
    Input('major_cat', 'hoverData'))

def update_min_cat_hover(hoverData):
    hover_country = 'Australia'
    
    if hoverData:
        hover_country = hoverData['points'][0]['customdata'][0]

    minor_cat_df = ecom_sales[ecom_sales['Country'] == hover_country]
    minor_cat_agg = minor_cat_df.groupby('Minor Category')['OrderValue'].agg('sum').reset_index(name='Total Sales ($)')
    ecom_bar_minor_cat = px.bar(minor_cat_agg, x='Total Sales ($)', y='Minor Category', orientation='h', height=450, width=480,title=f'Sales by Minor Category for: {hover_country}')
    ecom_bar_minor_cat.update_layout({'yaxis':{'dtick':1, 'categoryorder':'total ascending'}, 'title':{'x':0.5}})

    return ecom_bar_minor_cat


if __name__ == '__main__':
    app.run_server(debug=True)